### This is another script to tag articles. In this script I tagged companies in graph database and the locations. Since, tagging the resolved locations only was not a good idea, as at a number of places locaion mentioned in the article could be a city in a district or the name of the location may contain some prefix like "No

In [1]:
from pymongo import MongoClient

In [2]:
from bson.objectid import ObjectId

In [3]:
client = MongoClient(host='act4dgem.cse.iitd.ac.in')

In [4]:
local = MongoClient()

In [5]:
mediaDb = client.get_database('eventwise_media-db')

In [6]:
articles = local.get_database('media-db').get_collection('articlesWithGraphOrg2')

In [7]:
resolved_entities_1 = mediaDb.get_collection('all_media_entities_resolved_final')

In [8]:
resolved_entities_2 = mediaDb.get_collection('all_media_entities_resolved_final3')

In [9]:
articles.count()

34258

In [10]:
def updateText(objectId, text, entype, offset, length):
    if entype == 'Person':
        otag = '<PER>'
        ctag = '</PER>'
    elif entype == 'Company' or entype == 'Organization':
        otag = '<ORG>'
        ctag = '</ORG>'
    #elif entype == 'Country' or entype == 'City' or entype =='ProvinceOrState' or entype =='Continent':
    elif entype == 'City':
        otag = '<LOC>'
        ctag = '</LOC>'

    elif entype == 'ProvinceOrState':
        otag = '<STA>'
        ctag = '</STA>'
        
    else :
        otag = '<MISC>'
        ctag = '</MISC>'
    p1 = text[:offset]
    p2 = otag + str(objectId) + ctag
    p3 = text[offset+length:]

    shift = len(p2) - length

    return (p1 + p2 + p3, shift)

In [11]:
def updateOffset(entities, old_offset, shift):
    #print 'Updated Offset'
    for e in entities:
        for inst in e['instances']:
            #print inst
            if inst['offset'] > old_offset:
                inst['offset']+=shift
    return entities


In [12]:
def updateEntityData(entity):
    global articles,taggedArticles
    articleIds = entity['articleIds']
    names = [x.lower() for x in entity['aliases']]
    for articleId in articleIds:
        if articleId in taggedArticles:
            text = taggedArticles[articleId]['text']
            try:
                entities = taggedArticles[articleId]['entities']
            except:
                continue
        else:
            a = articles.find({'_id':ObjectId(articleId)}, no_cursor_timeout=True)
            if a.count()==0:
                #print("Article Not Found")
                #print(articleId)
                continue
            a = a[0]
            text = str(a['text'])
            try:
                entities = a['entities']
            except:
                continue
        for e in entities:
            if e['name'].lower() in names:
                for inst in e['instances']:
                    old_offset = inst['offset']
                    (text, shift) = updateText(entity['_id'], text, e['type'], inst['offset'], inst['length'])
                    entities = updateOffset(entities, old_offset, shift)
        if articleId not in taggedArticles:
            taggedArticles[articleId] = a    
        taggedArticles[articleId]['text'] = text
        taggedArticles[articleId]['entities'] = entities

In [13]:
def updateTextRaw(text, entype, offset, length):
    if entype == 'Person':
        otag = '<PER>'
        ctag = '</PER>'
    elif entype == 'Company' or entype == 'Organization':
        otag = '<ORG>'
        ctag = '</ORG>'
    elif entype == 'City':
        otag = '<LOC>'
        ctag = '</LOC>'
    elif entype == 'ProvinceOrState':
        otag = '<STA>'
        ctag = '</STA>'
    else :
        otag = '<MISC>'
        ctag = '</MISC>'
    p1 = text[:offset]
    p2 = otag + text[offset:offset+length] + ctag
    p3 = text[offset+length:]
    
    shift = len(p2) - length

    return (p1 + p2 + p3, shift)

In [14]:
resolved_entities_1.find().count()

2116075

In [15]:
resolved_entities_1.find({'type':{'$in':['City']}}).count()

15557

In [41]:
resolved_entities_2.find({'$and':[{'type':{'$in':['Company', 'Organization']}}, {'graphid':{'$exists':True}}]}).count()

94929

In [14]:
taggedArticles = dict()

In [17]:
# tagging person
for e in resolved_entities_1.find({'type':'Person'}):
    try:
        updateEntityData(e)
    except Exception as ex:
        print(ex)

In [15]:
for e in resolved_entities_2.find({'$and':[{'type':{'$in':['Company', 'Organization']}}, {'graphid':{'$exists':True}}]}):
    try:
        updateEntityData(e)
    except Exception as ex:
        print(ex)

In [16]:
# tagging all the locations mentioned in article without replacing with objectId

for aId in taggedArticles:
    article = taggedArticles[aId]
    text = article['text']
    ctr = 0
    entities = article['entities']
    city_offset = 0
    while ctr < len(entities):
        e = entities[ctr]
        type_ = e['type']
        if type_ == 'City':
            cityPresent = True
            name = e['name']
            for i in e['instances']:
                if e['type'] == 'City':
                    city_offset = 0
                leng = i['length']
                offset = i['offset']
                text, shift = updateTextRaw(text, type_, offset, leng)
                entities = updateOffset(entities, offset, shift)
        ctr += 1
    taggedArticles[aId]['text'] = text
    taggedArticles[aId]['entities'] = entities

In [17]:
c = 0
with open('taggedArticlesForLocatedIn.txt', 'w') as f:
    for a in taggedArticles:
        c += 1
        print(a, taggedArticles[a]['text'], file=f)
print(c)

34258


In [20]:
c = 0
with open('taggedArticleForWorksInWithArticleId.txt', 'w') as f:
    for a in taggedArticles:
        c += 1
        print(taggedArticles[a]['_id'], file=f)
        print(taggedArticles[a]['text'], file=f)
        print(file=f)
print(c)

34258


In [21]:
taggedArticlesCollection = local.get_database('media-db').get_collection('taggedArticlesWorksIn')
for a in taggedArticles:
    taggedArticlesCollection.insert_one(taggedArticles[a])

In [23]:
with open('taggedArticlesWithLocationAndOrg.txt', 'w') as f:
    for article in taggedArticlesCollection.find({"entities.type":"City"}):
        print(article['_id'], article['text'], '', sep='\n', file=f)